In [1]:
import numpy as np
import pandas as pd

import pyvinecopulib as pv
from vcimpute.generate_data import make_complete_data_matrix, mask_MCAR

In [2]:
controls = pv.FitControlsVinecop(family_set=[pv.BicopFamily.clayton], num_threads=10)

In [3]:
X = make_complete_data_matrix(1000, 5, 'clayton', vine_structure='R')

In [9]:
Xmis = mask_MCAR(X, mask_fraction=0.2, d_mis=2, monotonic_missingness=True)

In [54]:
while (mat.order[0] != 4) or (mat.order[1] != 5): # tmp
    mat = pv.RVineStructure.simulate(d=5)

In [55]:
from vcimpute.util import make_triangular_array
d=5
pcs = make_triangular_array(d)
for j in range(d-1):
    for i in range(d-j-1):
        pcs[i][j] = pv.Bicop()

In [56]:
cop = pv.Vinecop(structure=mat, pair_copulas=pcs)

In [57]:
cop.select(Xmis, controls=controls)

In [58]:
from vcimpute.simulator import simulate_order0, vfunc, find
from vcimpute.util import get
order0 = simulate_order0(cop, Xmis)
Ximp = np.copy(Xmis)
Ximp[np.isnan(Ximp[:, 4]), 4] = order0[np.isnan(Ximp[:, 4])]

In [ ]:
# 5|4,3,2,1 will not cover cases where 5,4 both missing

In [60]:
Ximp

array([[0.35961661, 0.36356717, 0.37102317, 0.35758188, 0.36624377],
       [0.03074983, 0.03065121, 0.03092155, 0.03105175, 0.03133021],
       [0.15384843, 0.16419719, 0.15644379,        nan,        nan],
       ...,
       [0.42958755, 0.44110131, 0.45889572, 0.42874444,        nan],
       [0.83487711, 0.84465445, 0.84597579, 0.81958847, 0.8267152 ],
       [0.2581658 , 0.25219484, 0.2432745 , 0.26288922, 0.24732774]])